In [34]:
import pandas as pd
import requests
from datetime import datetime

In [2]:
df = pd.read_csv('../files/stib_gtfs/stops.txt')

In [3]:
df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,0089,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,13.0
1,0470F,NaN,SIMONIS,NaN,50.863666,4.329612,NaN,NaN,0,NaN
2,0471,NaN,SIMONIS,NaN,50.863732,4.329236,NaN,NaN,0,39.0
3,0472,NaN,SIMONIS,NaN,50.863543,4.329023,NaN,NaN,0,39.0
4,0473F,NaN,SIMONIS,NaN,50.863418,4.330031,NaN,NaN,0,NaN


In [4]:
df['stop_name'] = [stop.lower() for stop in df['stop_name']]

In [5]:
stations = set(df['stop_name'])

In [17]:
def search_station(name, strict=False):
    if strict:
        return {el for el in stations if name.lower() == el}
    else:
        return {el for el in stations if name.lower() in el}

In [18]:
def get_stations_ids(names):
    ids = set()
    for name in names:
        ids.update(df[df['stop_name'] == name]['stop_id'])
    
    return ids

In [61]:
ids = get_stations_ids(s := search_station('hero', strict=False))
print(s, ids)

{'heros'} {'4076', '5058', '2355', '5058F', '1951', '5026', '1651', '4075'}


In [62]:
nexts = []

for id_ in ids:
    res = requests.get(f'http://127.0.0.1:8000/stib/{id_}').json()
    print(res)
    for point in res['points']:
        for value in point['passingTimes']:
            try:
                nexts.append({
                    'to': value['destination']['fr'],
                    'time': datetime.strptime(value['expectedArrivalTime'], '%Y-%m-%dT%H:%M:%S%z'),
                    'line': value['lineId']
                })
            except Exception as e:
                print(e.with_trackback())

{'points': [{'passingTimes': [{'destination': {'fr': 'GARE CENTRALE', 'nl': 'CENTRAAL STATION'}, 'expectedArrivalTime': '2021-10-14T18:19:00+02:00', 'lineId': '38'}, {'destination': {'fr': 'TRANSVAAL', 'nl': 'TRANSVAAL'}, 'expectedArrivalTime': '2021-10-14T18:18:00+02:00', 'lineId': '41'}, {'destination': {'fr': 'GARE CENTRALE', 'nl': 'CENTRAAL STATION'}, 'expectedArrivalTime': '2021-10-14T18:25:00+02:00', 'lineId': '38'}, {'destination': {'fr': 'TRANSVAAL', 'nl': 'TRANSVAAL'}, 'expectedArrivalTime': '2021-10-14T18:29:00+02:00', 'lineId': '41'}], 'pointId': '4076'}]}
{'points': [{'passingTimes': [{'destination': {'fr': 'GARE DU NORD', 'nl': 'NOORDSTATION'}, 'expectedArrivalTime': '2021-10-14T18:20:00+02:00', 'lineId': '4'}, {'destination': {'fr': 'STADE', 'nl': 'STADION'}, 'expectedArrivalTime': '2021-10-14T18:22:00+02:00', 'lineId': '51'}, {'destination': {'fr': 'ALBERT', 'nl': 'ALBERT'}, 'expectedArrivalTime': '2021-10-14T18:30:00+02:00', 'lineId': '70'}, {'destination': {'fr': 'SCHA

In [63]:
(nextdf := pd.DataFrame(nexts))

,to,time,line
0,GARE CENTRALE,2021-10-14 18:19:00+02:00,38
1,TRANSVAAL,2021-10-14 18:18:00+02:00,41
2,GARE CENTRALE,2021-10-14 18:25:00+02:00,38
3,TRANSVAAL,2021-10-14 18:29:00+02:00,41
4,GARE DU NORD,2021-10-14 18:20:00+02:00,4
5,STADE,2021-10-14 18:22:00+02:00,51
6,ALBERT,2021-10-14 18:30:00+02:00,70
7,SCHAERBEEK G.,2021-10-14 18:20:00+02:00,92
8,HEROS,2021-10-14 18:43:00+02:00,98
9,GARE DU NORD,2021-10-14 18:24:00+02:00,4
